<a href="https://colab.research.google.com/github/icees8/Speech-Understanding-Programming-Assignment-2/blob/main/Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Question 2. Source Separation

## Goal:
The goal of speech separation is to estimate individual speaker signals from their mixture, where the
source signals may be overlapped with each other entirely or partially.

## Task 1

Generate the LibriMix dataset by combining two speakers from the LibriSpeech dataset, focusing
solely on the LibriSpeech test clean partition. Take help from this [GitHub repo](https://github.com/JorisCos/LibriMix).

In [2]:
def get_librispeech_test_clean():
    '''downloads the LibriSpeech test clean partition'''
    !wget -c --tries=0 --read-timeout=20 http://www.openslr.org/resources/12/test-clean.tar.gz -P "/"
    !tar -xzf "test-clean.tar.gz" -C "/content/"
    !rm -rf test-clean.tar.gz

get_librispeech_test_clean()

--2024-04-05 05:41:25--  http://www.openslr.org/resources/12/test-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://openslr.elda.org/resources/12/test-clean.tar.gz [following]
--2024-04-05 05:41:25--  http://openslr.elda.org/resources/12/test-clean.tar.gz
Resolving openslr.elda.org (openslr.elda.org)... 141.94.109.138, 2001:41d0:203:ad8a::
Connecting to openslr.elda.org (openslr.elda.org)|141.94.109.138|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346663984 (331M) [application/x-gzip]
Saving to: ‘test-clean.tar.gz’

est-clean.tar.gz    100%[===================>] 330.60M  29.3MB/s    in 12s     

2024-04-05 05:41:38 (27.3 MB/s) - ‘test-clean.tar.gz’ saved [346663984/346663984]



In [3]:
def get_wham():
    '''Downloads the WHAM dataset'''
    !wget -c --tries=0 --read-timeout=20 https://my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com/wham_noise.zip -P "/"
    !unzip -qn wham_noise.zip -d "/content/"
    !rm -rf wham_noise.zip

get_wham()

--2024-04-05 05:41:42--  https://my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com/wham_noise.zip
Resolving my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com (my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com)... 3.5.25.182, 16.182.74.177, 16.182.67.177, ...
Connecting to my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com (my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com)|3.5.25.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18194930748 (17G) [application/zip]
Saving to: ‘wham_noise.zip’

ham_noise.zip        32%[=====>              ]   5.56G  31.4MB/s    eta 6m 44s ^C
[wham_noise.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of wham_noise.zip or
     

In [4]:
def get_librimix_repo():
    '''clones the librimix repo to current directory and install requirements'''
    !git clone https://github.com/JorisCos/LibriMix
    !pip install -r LibriMix/requirements.txt

get_librimix_repo()

fatal: destination path 'LibriMix' already exists and is not an empty directory.


In [5]:
def remove_file(path):
    !rm {path}

dir_path = "/content/LibriMix/metadata/Libri2Mix"

for file in os.listdir(dir_path):
    if "libri2mix_test-clean" not in file:
        remove_file(dir_path + "/" + file)

In [6]:
metadata_dir = "LibriMix/metadata/Libri2Mix"

!mkdir librimix
!python LibriMix/scripts/create_librimix_from_metadata.py --librispeech_dir "LibriSpeech" \
    --wham_dir "/content/wham_noise" \
    --metadata_dir {metadata_dir} \
    --librimix_outdir "librimix" \
    --n_src 2 \
    --freqs 8k \
    --modes max \
    --types mix_clean mix_single

mkdir: cannot create directory ‘librimix’: File exists
Directory librimix/Libri2Mix/wav8k/max/test already exist. Files won't be overwritten


In [7]:
!rm -r wham_noise
!rm -r LibriSpeech

rm: cannot remove 'wham_noise': No such file or directory


In [8]:
import pandas as pd

csv_path1 = "/content/librimix/Libri2Mix/wav8k/max/metadata/metrics_test_mix_clean.csv"
csv_path2 = "/content/librimix/Libri2Mix/wav8k/max/metadata/mixture_test_mix_clean.csv"

dataframe1 = pd.read_csv(csv_path1)
dataframe2 = pd.read_csv(csv_path2)

KeyboardInterrupt: 

In [ ]:
dataframe2

## Task 2

Partition the resulting LibriMix dataset into a 70-30 split for training and testing purposes. Evaluate

the performance of the pre-trained SepFormer on the testing set, employing scale-invariant signal-
to-noise ratio improvement (SISNRi) and signal-to-distortion ratio improvement (SDRi) as metrics.

For metric computation, consult the provided paper and utilize the code from torchmetrics.

## splitting dataset

splitting librimix dataset into 70:30 ratio

In [ ]:
librimix_csv_path = "/content/librimix/Libri2Mix/wav8k/max/metadata/mixture_test_mix_clean.csv"

librimix_csv = pd.read_csv(librimix_csv_path)

In [ ]:
from sklearn.model_selection import train_test_split

train_set , test_set = train_test_split(librimix_csv, train_size = 0.7, random_state = 42)

## installing libraries

In [1]:
!pip install pydub

In [2]:
!pip install wandb

In [3]:
!pip install torchmetrics

In [4]:
!pip install speechbrain

## loading model & wandb

In [5]:
from speechbrain.inference.separation import SepformerSeparation as separator
import torchaudio
import torch

In [6]:
device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)

In [7]:
model = separator.from_hparams(source="speechbrain/sepformer-whamr", savedir='pretrained_models/sepformer-whamr', run_opts={"device":device_name})
model = model.to(device)

## loading wandb

In [ ]:
import wandb

wandb.login()

## Utils & metrics

In [8]:
import librosa
import numpy as np

from torchmetrics.audio import PermutationInvariantTraining

from torchmetrics.functional.audio import scale_invariant_signal_noise_ratio
from torchmetrics.functional.audio import signal_distortion_ratio

from itertools import permutations

from speechbrain.nnet.losses import PitWrapper

In [9]:
from pydub import effects

def normalize(arr):
    return librosa.util.normalize(arr)

def load_audio(path):
    y_8k , sr = librosa.load(path, sr = 8000)
    y_8k = normalize(y_8k)
    return torch.tensor(y_8k).unsqueeze(0)

# sr1 = load_audio(dataframe2.iloc[1].source_1_path)
# sr2 = load_audio(dataframe2.iloc[1].source_2_path)
# mix = load_audio(dataframe2.iloc[1].mixture_path)

In [ ]:
# value = model.separate_batch(mix)

In [10]:
class SI_SNR_i:

    def __init__(self,**kwargs):

        self.sisnr = PermutationInvariantTraining(scale_invariant_signal_noise_ratio, mode="speaker-wise", eval_func = 'max')

    def __call__(self, mixed_signal, predicted_signals, target_signals):

        self.sisnr.to(target_signals.device)
        mixed_signal = torch.concat([mixed_signal,mixed_signal],dim=2).to(target_signals.device)

        predicted_signals   = predicted_signals.permute((0,2,1))
        target_signals      = target_signals.permute((0,2,1))
        mixed_signal        = mixed_signal.permute((0,2,1))

        sisnr_pred = self.sisnr(predicted_signals , target_signals)
        sisnr_base = self.sisnr(mixed_signal , target_signals)

        sisnr_i = sisnr_pred - sisnr_base

        return sisnr_i

In [11]:
class SDR_i:

    def __init__(self,**kwargs):

        self.sdr = PermutationInvariantTraining(signal_distortion_ratio, mode="speaker-wise", eval_func = 'max')

    def __call__(self, mixed_signal, predicted_signals, target_signals):

        self.sdr.to(target_signals.device)
        mixed_signal = torch.concat([mixed_signal,mixed_signal],dim=2).to(target_signals.device)

        predicted_signals   = predicted_signals.permute((0,2,1))
        target_signals      = target_signals.permute((0,2,1))
        mixed_signal        = mixed_signal.permute((0,2,1))

        sdr_pred = self.sdr(predicted_signals , target_signals)
        sdr_base = self.sdr(mixed_signal , target_signals)

        sdr_i = sdr_pred - sdr_base

        return sdr_i

In [ ]:
metric1 = SDR_i()
metric2 = SI_SNR_i()

sr1 = load_audio(train_set.iloc[1].source_1_path)
sr2 = load_audio(train_set.iloc[1].source_2_path)
mix = load_audio(train_set.iloc[1].mixture_path)

output = torch.stack([sr1,sr2],dim=-1)
target = torch.stack([sr1,sr2],dim=-1)
mix    = mix.unsqueeze(-1)

print(output.shape , target.shape, mix.shape)

print(metric1(mix,output,target))
print(metric2(mix,output,target))

In [17]:
from tqdm.notebook import tqdm

def evaluate_model(model , csv):

    sisnri  = []
    sdri    = []

    sisnri_func = SI_SNR_i()
    sdri_func   = SDR_i()

    for i , row in tqdm(list(csv.iterrows())):

        sr1 = load_audio(row.source_1_path)
        sr2 = load_audio(row.source_2_path)
        mix = load_audio(row.mixture_path)

        sr1 = sr1.to(device)
        sr2 = sr2.to(device)
        mix = mix.to(device)

        with torch.no_grad():
            output = model(mix)

        target = torch.stack([sr1,sr2],dim=2)
        mix = mix.unsqueeze(-1)

        sisnri.append((sisnri_func(mix , output , target)).detach().cpu().tolist())
        sdri.append((sdri_func(mix, output , target)).detach().cpu().tolist())

    # print(sisnri)
    # print(sdri)

    return np.mean(sisnri) , np.mean(sdri)

evaluate_model(model , test_set)

NameError: name 'test_set' is not defined

In [ ]:
[type(i) for i in model.parameters()]

In [12]:
2+2

4

In [15]:
EPOCHS = 5

from speechbrain.nnet.losses import get_si_snr_with_pitwrapper as sisnr_loss
from tqdm.notebook import tqdm

optimizer  = torch.optim.Adam(model.parameters(), lr = 15e-5)
criterion  = sisnr_loss

clip_grad_norm = 5
loss_upper_lim = 999999

from torch.cuda.amp import GradScaler
scaler = GradScaler()

model.train()

for param in model.parameters():
    param.requires_grad = True

# run = wandb.init(
#     project="Q2_training",
#     config={
#         "learning_rate": optimizer.param_groups[-1]['lr'],
#         "epochs": EPOCHS,
#     },
# )

for epoch in range(EPOCHS):

    sisnri  = []
    sdri    = []
    loss    = []

    sisnri_func = SI_SNR_i()
    sdri_func   = SDR_i()

    nonfinite_count = 0

    with torch.autocast(dtype=torch.float16, device_type=torch.device(device).type):
        for i , row in tqdm(train_set.iterrows()):
        # for i , row in tqdm(enumerate(range(10))):

            # sr1 = load_audio(row.source_1_path)
            # sr2 = load_audio(row.source_2_path)
            # mix = load_audio(row.mixture_path)

            # sr1 = torch.rand(1,80000)
            # sr2 = torch.rand(1,80000)
            # mix = torch.rand(1,80000)

            sr1 = sr1.to(device)
            sr2 = sr2.to(device)
            mix = mix.to(device)

            optimizer.zero_grad(set_to_none = True)

            output = model(mix)
            target = torch.stack([sr1,sr2],dim=2)

            loss_value = criterion(output , target)

            loss_value = loss_value[loss_value > -30]
            if loss_value.nelement() > 0:
                loss_value = loss_value.mean()

            if (loss_value.nelement() > 0 and loss_value < loss_upper_lim):
                scaler.scale(loss_value).backward()
                if clip_grad_norm >= 0:
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(
                        model.parameters(),
                        clip_grad_norm,
                    )
                scaler.step(optimizer)
                scaler.update()
            else:
                nonfinite_count += 1
                loss_value.data = torch.tensor(0).to(device)

        mix = mix.unsqueeze(-1)

        loss.append((loss_value))
        sisnri.append((sisnri_func(mix , output , target)))
        sdri.append((sdri_func(mix, output , target)))

    # dataset_size = len(train_set)
    # print(dataset_size)

    epoch_loss = loss

optimizer.zero_grad(set_to_none = True)


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]